# Корелляции  

Задача: 

**№1.** Используя язык Python, необходимо запрогроммировать модифицированные корреляции для тяжелых и сверхтяжелых нефтей. 

**№2** Построить зависимости вязкости газонасыщенной нефти, газосодержания, сжимаемости и пластовой температуры для различных температур. Исходные данные: газовый фактор, пластовая температура, относительная плотность нефти, относительная плотность газа, давление в сепараторе.

## Границы применимости корреляций


  
  
| Параметр      | Обозначение              | Значение | Размерность |
| :------------- |:------------------:| :-----:| :---------:|
| Плотность нефти     | $$API$$   | [6;22.3] | $$API^0$$ |
| Пластовое давление   | $$P_r$$ |  [1038.49;7411.54] | $$psia$$  |
| Пластовая температура  | $$T_r$$ |  [131.4;250.7] | $$F^0$$  |
| Газосодержание нефти в пластовых условиях    | $$R_s$$   | [17.21;640.25] | $$\frac{scf}{STB}$$ |
| Давление насыщения   | $$API$$   | [208.86;4021.96] | $$psia$$ |
| Давление в сепараторе  | $$API$$   | [14.5;752.2] | $$psia$$ |
| Температура в сепараторе  | $$T_{sp}$$   | [59;177.8] | $$F^0$$ |
| Газосодержание в сепараторе     | $$R_{sp}$$   | [11.1;575.62] | $$\frac{scf}{STB}$$ |
| Газосодержание в поверхностных условиях     | $$R_{st}$$   | [4.39;311.41] | $$\frac{scf}{STB}$$ | 
| Относительная плотность газа на поверхности     | $$\gamma_{g}$$   | [0.623;1.517] | $$$$ |
| Относительная плотность газа в сепараторе     | $$\gamma_{Psp}$$   | [0.623;1.517] | $$$$ |
| Мольная концентрация углекислого газа     | $$YCO2$$   | [0.5;98.8] | _%mol._ |
| Мольная концентрация азота    | $$YN2$$   | [0;63.32] | _%mol._ |
| Мольная концентрация сероводорода    | $$YH2S$$   | [0;1.99] | _%mol._ |
| Объемный коэффициент нефти   | $$OFVF$$   | [1.057;1.362] | $$\frac{bbl}{STB}$$ |
| Cжимаемость нефти    | $$C_o$$   | [3.02;42.9] |  $$psia^{-1} \bullet 10^6$$ |
| Вязкость дегазированной нефти    | $$ \mu_{od}$$   | [7.7;1386.9] | $$cp$$ |
| Вязкость насыщенной нефти    | $$ \mu_{ol}$$   | [2.1;295.9] | $$cp$$ |
| Вязкость недонасыщенной нефти    | $$ \mu_{o}$$   | [2.4;354.6] | $$cp$$ |








## Пункт №1

**Прогроммируем модифицированные корреляции:**

1.Для вычисления давления насыщения в работе используется **Moddified Standing's correlation**:

$$P_b=15.7286\bullet \left[ (\frac{R_s}{\gamma_g}) \bullet \frac{10^{0.0020\bullet T}}{10^{0.0142\bullet API}} \right]$$

Корреляция применима для тяжелых нефтей ($ 10< API^\circ \le22.3 $).

In [2]:
def Bubblepoint_P_HO(Rs, Yg, T, API):
    if 10 < API <= 22.3:
        Pb = 15.7286*((Rs/Yg)**0.7885)*10**(0.0020*T-0.0142*API)
        return Pb
    
    else:
        return 'Корреляция не подходит для данного значения плотности'


print(Bubblepoint_P_HO(231.46,0.696,147.9,11.0))

2112.4282561966543


2.Газонасыщенность:
>Cверхтяжелых нефтей вычисляют с помощью **Modified Standing's correlation**:
$$R_s=\gamma_g\bullet \left(\frac{P_b}{10.7025}\bullet10^{(0.0169\bullet API-0.00156\bullet T)}\right)^{1.1128} $$
>Тяжелых нефтей с помощью **Modifiend Vasquez-Beggs correlation**:
$$R_s= \frac{\gamma_{gcorr} \bullet P_b^{1.2057}}{56.434}\bullet 10^{10.9267\bullet API/(T+460)}$$
>Где:
$$\gamma_{gcorr}=\gamma_{gPsp} \bullet \left[1+0.5912 \bullet API \bullet T_{sp} \bullet Log(\frac{P_{sp}}{114.7})10^{-4} \right]$$

In [3]:
import math as m


def Solution_GOR(Pb, Yg, T, API, Ygpsp, Tsp, Psp):
    if API <= 10:
        Rs = Yg*(Pb/10.7025*10**(0.0169*API-0.00156*T))**1.1128
        return Rs
    
    elif 10 < API <= 22.3:
        Ygcorr = Ygpsp*(1+0.5912*API*Tsp*m.log10(Psp/114.7)*10**(-4))
        Rs = Ygcorr*Pb**(1.2057)/56.434*10.9267*(API/(T+460))
        return Rs
    
    else:
        return 'Корреляция не подходит для данного значения плотности'

print(Solution_GOR(2503.39,0.696,147.9,22,0.679,131.0,58.02))

56.564085705371255


3.Сжимаемость для сверхтяжелых нефтей вычисляют с помощью **Modifiend Vasquez-Beggs correlation**:
$$C_o= \frac{-889.6+3.1374\bullet R_s+20 \bullet T_g-627.3 \bullet \gamma_{gcorr}+41.91 \bullet API}{P_g \bullet 10^5}$$
>Где:
$$\gamma_{gcorr}=\gamma_{gPsp} \bullet \left[1+0.5912 \bullet API \bullet T_{sp} \bullet Log(\frac{P_{sp}}{114.7})10^{-4} \right]$$

Сжимаемость для тяжелых нефтей вычисляют с помощью **Modifiend Vasquez-Beggs correlation**:
$$C_o= \frac{-2841.8+2.9646\bullet R_s+25.5439 \bullet T_g-1230.5 \bullet \gamma_{gcorr}+41.91 \bullet API}{P_g \bullet 10^5}$$
>Где:
$$\gamma_{gcorr}=\gamma_{gPsp} \bullet \left[1+0.5912 \bullet API \bullet T_{sp} \bullet Log(\frac{P_{sp}}{114.7})10^{-4} \right]$$

In [13]:
import math as m


def Isotherm_Compressibility(Rs, Tg, Pg, API, Ygpsp, Tsp, Psp):
    if API <= 10:
        Ygcorr = Ygpsp*(1+0.5912*API*Tsp*m.log10(Psp/114.7)*10**(-4))
        Co = (-889.6+3.1374*Rs+20*Tg-627.3*Ygcorr-81.4476*API)/(Pg*10**(5))
        return Co
    
    elif 10 < API <=  22.3:
        Ygcorr = Ygpsp*(1+0.5912*API*Tsp*m.log10(Psp/114.7)*10**(-4))
        Co = (-2841.8+2.9646*Rs+25.5439*Tg-1230.5*Ygcorr+41.91*API)/(Pg*10**(5))
        return Co
    
    else:
        return 'Корреляция не подходит для данного значения плотности'

print(Isotherm_Compressibility(2503.39,0.696,147.9,6.0,0.679,131.0,58.0))

0.0004103918029077642


4.Вязкость дегазированной нефти в находят с помощью следующих корреляций:
>**Modifiend Egbogah-Jack's correlation** для сверхтяжелых нефтей:
$$log\bullet log(\mu_{od} +1)=1.90296-0.012619\bullet API -0.61748 \bullet log(T_g)$$
>**Modifiend Egbogah-Jack's correlation** для тяжелых нефтей:
$$log\bullet log(\mu_{od} +1)=2.06492-0.0179\bullet API -0.70226 \bullet log(T_g)$$

In [5]:
def D_visc(API, T):
    if API <= 10:
        Mod = 10**(10**(1.90296-0.012619*API-0.61748*m.log10(T)))-1
        return Mod
     
    elif 10 < API <= 22.3:
        Mod = 10**(10**(2.06492-0.0179*API-0.70226*m.log10(T)))-1
        return Mod
    
    else:
        return 'Корреляция не подходит для данного значения плотности'

print(D_visc(6.0,147.9))

1177.646405184529


5.Вязкость насыщенной нефти в находят с помощью следующих корреляций:
>**Modifiend Kartoatmodjo's correlation** для сверхтяжелых нефтей:
$$\mu_{ol}=2.3945+0.8927\bullet F+0.001567 \bullet F^2$$
Где
$$F= \left(-0.0335+1.0785\bullet10^{-0.000845\bullet R_s}\right)\bullet\mu_{od}^{(0.5798+0.3432\bullet y)}$$
$$y=10^{-0.00081\bullet R_s}$$
$$\gamma_{gcorr}=\gamma_{gPsp}\bullet \left[1+0.1595\bullet API^{0.4078} \bullet (T_{sp})^{-0.2466}\bullet Log(\frac{P_{sp}}{114.7})\right]$$


>**Modifiend Kartoatmodjo's correlation** для тяжелых нефтей:
$$\mu_{ol}=-0.6311+1.078\bullet F-0.003653 \bullet F^2$$
Где
$$F= \left(0.2478+0.6114\bullet10^{-0.000845\bullet R_s}\right)\bullet\mu_{od}^{(0.4731+0.5158\bullet y)}$$
$$y=10^{-0.00081\bullet R_s}$$
$$\gamma_{gcorr}=\gamma_{gPsp}\bullet \left[1+0.1595\bullet API^{0.4078} \bullet (T_{sp})^{-0.2466}\bullet Log(\frac{P_{sp}}{114.7})\right]$$

In [6]:
def G_S_visc(Rs, Mod, API, Ygpsp, Tsp, Psp):
    if API <= 10:
        Ygcorr = Ygpsp*(1+0.1595*API**(0.4078)*Tsp**(-0.2466)*m.log10(Psp/114.7))
        Y = 10**(-0.00081*Rs)
        F = (-0.0335+1.0785*10**(-0.000845*Rs))*Mod**(0.5798+0.3432*Y)
        Mol = 2.3945+0.8927*F+0.001567*F**2
        return Mol
    elif 10 < API <= 22.3:
        Ygcorr = Ygpsp*(1+0.1595*API**(0.4078)*Tsp**(-0.2466)*m.log10(Psp/114.7))
        Y = 10**(-0.00081*Rs)
        F = (0.2478+0.6114*10**(-0.000845*Rs))*Mod**(0.4731+0.5158*Y)
        Mol = -0.6311+1.078*F-0.003653*F**2
        return Mol
    else:
        return 'Корреляция не подходит для данного значения плотности'

print(G_S_visc(231.46,1386.9,6.0,0.679,131.0,58.02))

270.78565954057524


6.Вязкость недонасыщенной нефти в находят с помощью следующих корреляций:
>**Modifiend Labedi's correlation** для сверхтяжелых нефтей:
$$\mu_o=\mu_{ol}-\left[(1- \frac{P}{P_b})\bullet \left(\frac{10^{-2.19}\bullet \mu_{od}^{1.055}\bullet P_b^{0.3132}}{10^{0.0099\bullet API}}\right)\right]$$



>**Modifiend Kartoatmodjo's correlation** для тяжелых нефтей:
$$\mu_{o}=0.9886\bullet \mu_{ol}+0.002763\bullet(P-P_b)\bullet(-0.01153\bullet \mu_{ol}^{1.7933}+0.0316\bullet \mu_{ol}^{1.5939})$$
Где

$$\gamma_{gcorr}=\gamma_{gPsp}\bullet \left[1+0.1595\bullet API^{0.4078} \bullet (T_{sp})^{-0.2466}\bullet Log(\frac{P_{sp}}{114.7})\right]$$

In [19]:
def Un_S_visc(Mol, Mod, P, Pb, API, Ygpsp, Tsp, Psp):
    if API <= 10:
        Mo = Mol-((1-P/Pb)*10**(-2.19)*Mod**(1.055)*Pb**(0.3132)/10**(0.0099)/API)
        return Mo
    elif 10 < API <= 22.3:
        Ygcorr = Ygpsp*(1+0.1595*API**(0.4078)*Tsp**(-0.2466)*m.log10(Psp/114.7))
        Mo = 0.9886*Mol+0.002763*(P-Pb)*(-0.01153*Mol**(1.7933)+0.0316*Mol**(1.5939))
        return Mo
    else:
        return 'Корреляция не подходит для данного значения плотности'

    
Un_S_visc(295.9,1386.9,3428.75,2503.39,6.0,0.679,131.0,58.02)

print(Un_S_visc(295.9,1386.9,3428.75,2503.39,6.0,0.679,131.0,58.02))

305.2112442848726


**Сравнение результатов расчетов с данными из статьи по образцу №1 показало хорошую сходимость результатов.** 

# Пункт №2
Построим зависимости для t = 30 °С; 50 °С; 70 °С; 90 °С:

𝜇𝑜 = 𝑓(𝑝,𝑡 = 𝑐𝑜𝑛𝑠𝑡); 𝑝𝑏 = 𝑓(𝑝,𝑡 = 𝑐𝑜𝑛𝑠𝑡); 𝑅𝑠 = 𝑓(𝑝,𝑡 = 𝑐𝑜𝑛𝑠𝑡); 𝑐𝑜 = 𝑓(𝑝,𝑡 = 𝑐𝑜𝑛𝑠𝑡)

где: 𝜇𝑜 – вязкость газонасыщенной нефти, 𝑝𝑏 – давление насыщения; 𝑅𝑠 – газосодержание; 𝑐𝑜 – сжимаемость нефти

Графики построим при помощи библиотеки plotly.

Учтем, что для t = 30 °С; 50 °С расчеты могут быть некорректны, т.к. данные значения находятся за пределами границ применимости модиффицированных корреляций,для наглядности их визуализировать не будем.

In [1]:
import math as m
import plotly
import plotly.graph_objs as go
import plotly.express as px

class Converter:
    """
    Класс для конвертации
    """
    def convert_t_fromsi_to_F(self, t_c: float) -> float: 
        t_f = t_c * 1.8 +32
        return t_f

    def convert_rs_fromsi_to_scfSTB(self,rs_m3m3) -> float:
        rs_imp = rs_m3m3 * 5.5525
        return rs_imp
    
    def convert_fromatm_to_psia(self,p_atm) -> float:
        p_psia = 14.696 * p_atm
        return p_psia
    
    def convert_sg_to_api(self,sg) -> float:
        api = 141.5 / sg -131.5
        return api

    def convert_t_fromF_to_si(self,t_f: float) -> float:
        t_c = (t_f - 32) / 1.8
        return t_c
    
    def convert_rs_fromscfSTB_to_si(self,rs_imp) -> float:
        rs_m3m3 = rs_imp * 0.1801
        return rs_m3m3
    
    def convert_frompsia_to_atm(self,p_psia) -> float:
        p_atm = p_psia * 0.06804
        return p_atm
    

    


class Fluid(Converter):
    """
    Класс для расчета свойств флюида
    """
    def __init__(self, rs_m3m3 = 33.64, yg: float = 0.696, t_c:float = 70, yg_psp: float = 0.679, p_sp_atm: float = 3.65, sg:float = 0.966,
                 p_atm: float = 30, pb_atm: float = 30, t_sp_c: float = 1, mu_od: float = 111, mu_ol: float = 50) -> None:
        self.rs_m3m3 = rs_m3m3
        self.t_c = t_c
        self.t_sp_c = t_sp_c
        self.psp_atm = p_sp_atm
        self.p_atm = p_atm
        self.pb_atm = pb_atm
        self.yg = yg
        self.yg_psp = yg_psp
        self.sg = sg
        self.mu_od = mu_od
        self.mu_ol = mu_ol

        self.rs_imp = None
        self.t_f = None
        self.t_sp_f = None
        self.psp_psi = None
        self.p_psi = None
        self.pb_psi = None
        self.api = None
        
        self.convert_initdata()
         
        self.pb_r = self.calc_pb(self.rs_imp, self.yg, self.t_f, self.api)
        self.rs_r = self.calc_rs(self.pb_psi, self.yg, self.t_f, self.api, self.yg_psp, self.t_sp_f, self.psp_psi)
        self.co_r = self.calc_co(self.rs_imp, self.t_f, self.p_psi, self.api, self.yg_psp, self.t_sp_f, self.psp_psi)
        self.mu_odr = self.calc_muod(self.api, self.t_f)
        self.mu_olr = self.calc_muol(self.rs_imp, self.mu_od, self.api, self.yg_psp, self.t_sp_f, self.psp_psi)
        self.mu_r = self.calc_mo(self.mu_ol, self.mu_od, self.p_psi, self.pb_psi, self.api, yg_psp, self.t_sp_f, self.psp_psi)

    def convert_initdata(self) -> float :
        self.rs_imp = self.convert_rs_fromsi_to_scfSTB(self.rs_m3m3)
        self.t_f = self.convert_t_fromsi_to_F(self.t_c)
        self.t_sp_f = self.convert_t_fromsi_to_F(self.t_sp_c)
        self.psp_psi = self.convert_fromatm_to_psia(self.psp_atm)
        self.p_psi = self.convert_fromatm_to_psia(self.p_atm)
        self.pb_psi = self.convert_fromatm_to_psia(self.pb_atm)
        self.api = self.convert_sg_to_api(self.sg)
        
    def calc_pb(self, rs_imp, yg, t_f, api) -> float :
        Pb = 15.7286*((rs_imp/yg)**0.7885)*10**(0.0020*t_f-0.0142*api)
        return Pb * 0.06804

        #2
    def calc_rs(self, pb_psi, yg, t_f, api, yg_psp, t_sp_f, psp_psi) -> float :
        if api <= 10:
            Rs = yg * (pb_psi/10.7025*10**(0.0169*api-0.00156*t_f))**1.1128
            return Rs * 0.1801
        elif 10 < api <= 22.3:
            Ygcorr = yg_psp*(1+0.5912*api*t_sp_f*(m.log10(psp_psi/114.7))*10**(-4))
            Rs = Ygcorr*pb_psi**(1.2057)/56.434*10.9267*(api/(t_f+460))
            return Rs * 0.1801
        else:
            raise Exception
            # print('Корреляция не подходит для данного значения плотности')

    #3
    def calc_co(self, rs_imp, t_f, p_psi, api, yg_psp, t_sp_f, psp_psi) -> float :
        if api <= 10:
            Ygcorr = yg_psp* ( 1 + 0.5912 * api * t_sp_f * ( m.log10(psp_psi/114.7)) * 10**(-4))
            Co = (-889.6+3.1374*rs_imp+20*t_f-627.3*Ygcorr-81.4476*api)/(p_psi*10**(5))
            return Co * 14.696
        elif 10 < api <= 22.3:
            Ygcorr = yg_psp*(1 + 0.5912 * api*t_sp_f*(m.log10(psp_psi/114.7))*10**(-4))
            Co = (-2841.8+2.9646*rs_imp +25.5439*t_f-1230.5*Ygcorr+41.91*api)/(p_psi*10**(5))
            return Co * 14.696
        else:
            return 'Корреляция не подходит для данного значения плотности'

    #4
    def calc_muod(self, api, t_f) -> float :
        if api <= 10:
            Mod = 10**(10**(1.90296-0.012619*api-0.61748*m.log10(t_f)))-1
            return Mod
        elif 10 < api <= 22.3:
            Mod = 10**(10**(2.06492-0.0179*api-0.70226*m.log10(t_f)))-1
            return Mod
        else:
            return 'Корреляция не подходит для данного значения плотности'

    #5
    def calc_muol(self, rs_imp, mu_od, api, yg_psp, t_sp_f, p_sp_psi) -> float :
        if api <= 10:
            Ygcorr = yg_psp*(1+0.1595*api**(0.4078)*t_sp_f**(-0.2466)*m.log10(p_sp_psi/114.7))
            Y = 10**(-0.00081*rs_imp)
            F = (-0.0335+1.0785*10**(-0.000845*rs_imp))*mu_od**(0.5798+0.3432*Y)
            Mol = 2.3945+0.8927*F+0.001567*F**2
            return Mol
        elif 10 < api <= 22.3:
            Ygcorr = yg_psp*(1+0.1595*api**(0.4078)*t_sp_f**(-0.2466)*m.log10(p_sp_psi/114.7))
            Y = 10**(-0.00081*rs_imp)
            F = (0.2478+0.6114*10**(-0.000845*rs_imp))*mu_od**(0.4731+0.5158*Y)
            Mol = -0.6311+1.078*F-0.003653*F**2
            return Mol
        else:
            return 'Корреляция не подходит для данного значения плотности'

    #6
    def calc_mo(self, mu_ol, mu_od, p_psi, pb_psi, api, yg_psp, t_sp_f, p_sp_psi) -> float :
        if api <= 10:
            Mo = mu_ol -((1-p_psi/pb_psi)*10**(-2.19)*mu_od**(1.055)*pb_psi**(0.3132)/10**(0.0099)/api)
            return Mo
        elif 10 < api <= 22.3:
            Ygcorr = yg_psp*(1+0.1595*api**(0.4078)*t_sp_f**(-0.2466)*m.log10(p_sp_psi/114.7))
            Mo = 0.9886*mu_ol+0.002763*(p_psi-pb_psi)*(-0.01153*mu_ol**(1.7933)+0.0316*mu_ol**(1.5939))
            return Mo
        else:
            return 'Корреляция не подходит для данного значения плотности'

def calc_rs_list(p:list,t):
    rsr = []
    for i in p:
        rasc = Fluid(p_atm=i,t_c=t)
        rsr.append(rasc.rs_r)
    return rsr

def calc_muol_list(p,t):
    muol_r = []
    rasc = Fluid(t_c=t)
    muod_r = rasc.mu_odr
    rasc = Fluid(mu_od=muod_r)
    rs = calc_rs_list(p,t)
    for i in rs:
        rasc = Fluid(rs_m3m3=i)
        muol_r.append(rasc.mu_olr)
    return muol_r

def calc_co_list(p,t):
    co = []
    rs = calc_rs_list(p,t)
    for pr, rr in zip(p,rs):
        rasc = Fluid(rs_m3m3=rr,p_atm=pr,t_c=t)
        co.append(rasc.co_r)
    return co

if __name__ == '__main__':
    rasc = Fluid(t_c=70)
    pb = rasc.pb_r
    pb = m.ceil(pb)
    p_list = [i for i in range(1, pb, 1)]

    rs_30 = calc_rs_list(p=p_list, t = 30)
    rs_50 = calc_rs_list(p=p_list, t = 50)
    rs_70 = calc_rs_list(p=p_list, t = 70)
    rs_90 = calc_rs_list(p=p_list, t = 90)

    mu_ol_30 = calc_muol_list(p_list, t=30)
    mu_ol_50 = calc_muol_list(p_list, t=50)
    mu_ol_70 = calc_muol_list(p_list, t=70)
    mu_ol_90 = calc_muol_list(p_list, t=90)

    co_30 = calc_co_list(p_list, t=30)
    co_50 = calc_co_list(p_list, t=50)
    co_70 = calc_co_list(p_list, t=70)
    co_90 = calc_co_list(p_list, t=90)





    


**Построим графики зависимости газосодержания,вязкости насыщенной нефти и сжимаемости от давления.**

Графики будем строить до давления насыщения.

**Газонасыщенность с увеличением давления до давления насыщения растет, так как на газ и нефть действует все большее давление, способствующее растворению газа в нефти, а нефть до давления насыщения еще может растворить в себе газ. Газонасыщенность при увеличении температуры падает.**

In [151]:
#визуализация


print("Давление насыщения = " + str(Pn_si) + " атм.")

fig = go.Figure()

fig.add_trace(go.Scatter(x=P_si, y=Rs70_si, name='$$R_s(T=70^0 C)$$'))
fig.add_trace(go.Scatter(x=P_si, y=Rs90_si, name='$$R_s(T=90^0 C)$$'))
fig.update_layout(legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  title="Зависимость газосодержания от давления",
                  xaxis_title="Давление, атм",
                  yaxis_title="Газосодержание, м3/м3",
                  margin=dict(l=0, r=0, t=30, b=0))
fig.update_traces(hoverinfo="x+y")
fig.show()




Давление насыщения = 102.04966470651252 атм.


**С повышением давления до давления насыщения вязкость газонасыщенной нефти падает, так как в ней растворяется все больше газа. С повышением температуры вязкость нефти так же уменьшается.**

In [152]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(x=P_si, y=Mol70, name='$$M_{ol}(T=70^0 C)$$'))
fig1.add_trace(go.Scatter(x=P_si, y=Mol90, name='$$M_{ol}(T=90^0 C)$$'))
fig1.update_layout(legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  title="Зависимость вязкости газонасыщенной нефти от давления",
                  xaxis_title="Давление, атм",
                  yaxis_title="Вязкость, сП",
                  margin=dict(l=0, r=0, t=30, b=0))
fig1.update_traces(hoverinfo="x+y")
fig1.show()


**При повышении давления коэффициент сжимаемости падает(следствие уменьшения расстояния между частицами нефти). Чем выше температура, тем выше коэффициент сжимаемости нефти(следствие теплового расширения).**

In [153]:
fig2 = go.Figure()

fig2.add_trace(go.Scatter(x=P_si, y=Co70_si, name='$$C_o(T=70^0 C)$$'))
fig2.add_trace(go.Scatter(x=P_si, y=Co90_si, name='$$C_o(T=90^0 C)$$'))
fig2.update_layout(legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  title="Зависимость сжимаемости от давления",
                  xaxis_title="Давление, атм",
                  yaxis_title="Сжимаемость, 1/атм",
                  margin=dict(l=0, r=0, t=30, b=0))
fig2.update_traces(hoverinfo="x+y")
fig2.show()


**Сравнение вязкости и давление насыщения с данными по месторождению №1 из тех.режима показывает, что расчет в целом верен, значения имеют один порядок, но условия на месторождение(низкая температура) и недостаток данных по месторождению не позволяют провести расчет по данным из тех.режима.**